In [3]:
# Ontology factories are used to make ontologies from
# either local files or remote services
from ontobio import OntologyFactory
ofa = OntologyFactory()

In [4]:
# Use the makefile to generate an obojson version of ICD10
icd = ofa.create("icd10.json")

In [5]:
# Create HP ontology object using default method (remote services)
hp = ofa.create('hp')

In [44]:
# Make a simple version of HPO with no inter-ontology links
hpsimple = hp.subontology(relations='subClassOf')

In [85]:
# Read in the previously generated mappings
# (see Makefile for generation)
import pandas as pd
df=pd.read_csv('ontobio-align-hp.tsv',sep="\t")

In [86]:
# Show sample rows
df[0:10]

,left,left_label,right,right_label,left_match_type,right_match_type,left_match_val,right_match_val,score,left_simscore,right_simscore,reciprocal_score,conditional_pr_equiv,equiv_clique_size
0,http://purl.obolibrary.org/obo/ICD10_Q66.4,Congenital talipes calcaneovalgus,HP:0005850,Congenital talipes calcaneovalgus,label,label,Congenital talipes calcaneovalgus,Congenital talipes calcaneovalgus,100.00,0.5,1.000000,0,0.580552,3
1,http://purl.obolibrary.org/obo/ICD10_Q25.72,Congenital pulmonary arteriovenous malformation,HP:0006548,Pulmonary arteriovenous malformation,label,label,Congenital pulmonary arteriovenous malformation,Pulmonary arteriovenous malformation,72.25,0.5,1.000000,4,1.000000,2
2,http://purl.obolibrary.org/obo/ICD10_E26.1,Secondary hyperaldosteronism,HP:0011741,Secondary hyperaldosteronism,label,label,Secondary hyperaldosteronism,Secondary hyperaldosteronism,100.00,1.0,1.000000,4,1.000000,2
3,http://purl.obolibrary.org/obo/ICD10_G71.0,Muscular dystrophy,HP:0003560,Muscular dystrophy,label,label,Muscular dystrophy,Muscular dystrophy,100.00,1.0,0.500000,0,0.580552,3
4,http://purl.obolibrary.org/obo/ICD10_G71.0,Muscular dystrophy,HP:0003741,Congenital muscular dystrophy,label,label,Muscular dystrophy,Congenital muscular dystrophy,72.25,1.0,0.500000,2,0.419448,3
5,http://purl.obolibrary.org/obo/ICD10_N50.82,Scrotal pain,HP:0030155,Scrotal pain,label,label,Scrotal pain,Scrotal pain,100.00,1.0,0.500000,4,1.000000,2
6,http://purl.obolibrary.org/obo/ICD10_Q54.1,"Hypospadias, penile",HP:0003244,Penile hypospadias,label,label,"Hypospadias, penile",Penile hypospadias,72.25,1.0,0.666667,4,1.000000,2
7,http://purl.obolibrary.org/obo/ICD10_R63.0,Anorexia,HP:0002039,Anorexia,label,label,Anorexia,Anorexia,100.00,1.0,1.000000,4,1.000000,2
8,http://purl.obolibrary.org/obo/ICD10_Q52.11,Transverse vaginal septum,HP:0000145,Transverse vaginal septum,label,label,Transverse vaginal septum,Transverse vaginal septum,100.00,1.0,1.000000,4,1.000000,2
9,http://purl.obolibrary.org/obo/ICD10_H57.02,Anisocoria,HP:0009916,Anisocoria,label,label,Anisocoria,Anisocoria,100.00,0.5,1.000000,4,1.000000,2


In [87]:
## Query by row
svas = df.loc[df['right'] == 'HP:0004381']
svas

,left,left_label,right,right_label,left_match_type,right_match_type,left_match_val,right_match_val,score,left_simscore,right_simscore,reciprocal_score,conditional_pr_equiv,equiv_clique_size
596,http://purl.obolibrary.org/obo/ICD10_Q25.3,Supravalvular aortic stenosis,HP:0004381,Supravalvular aortic stenosis,label,label,Supravalvular aortic stenosis,Supravalvular aortic stenosis,100.0,0.5,1.0,4,1.0,2


In [88]:
## Make a mapping
from collections import defaultdict
hp2icd = defaultdict(list)
for _,row in df.iterrows():
    hp2icd[row['right']].append(row['left'])
    
# test it
hp2icd['HP:0004381']

['http://purl.obolibrary.org/obo/ICD10_Q25.3']

In [89]:
## Use associationset factory to make assocset from
## Remote services (Monarch)
from ontobio import AssociationSetFactory
afa = AssociationSetFactory()

In [79]:
aset = afa.create(ontology=hp, subject_category='disease', object_category='phenotype', taxon='NCBITaxon:9606')

In [90]:
## rollup an HP term.
## Algorithm:
##  use direct mapping if available
##  otherwise recursively call on all parents, and take union of results
def rollup(hpterm):
    mterms = set(hp2icd[hpterm])
    if len(mterms) > 0:
        return mterms
    for p in hpsimple.parents(hpterm):
        #print("P {} ->{}".format(hpterm,p))
        mterms.update(rollup(p))
    return mterms

def test_rollup(t):
    print("ROLLING UP: {} {}".format(t,hp.label(t)))
    terms = rollup(t)
    print(" --> {}".format( ["{} {}".format(x,icd.label(x)) for x in terms] ))
    
# test on one with direct mapping
test_rollup('HP:0004381')

# test on one with no direct mapping;
# we expect a mapping for the parent
test_rollup('HP:0005173')

ROLLING UP: HP:0004381 Supravalvular aortic stenosis
 --> ['http://purl.obolibrary.org/obo/ICD10_Q25.3 Supravalvular aortic stenosis']
ROLLING UP: HP:0005173 Calcific aortic valve stenosis
 --> ['http://purl.obolibrary.org/obo/ICD10_Q23.0 Congenital stenosis of aortic valve']


In [92]:
## Create disease-to-ICD10 mappings
d2icd = {}
for d in aset.subjects:
    hpterms = aset.annotations(d)
    mterms = set()
    for t in hpterms:
        mterms.update(rollup(t))
    d2icd[d] = mterms
    

In [93]:
## Test it
def show_phekb(d):
    print("\nTESTING: {} {}".format(d,aset.label(d)))
    print("  HPO PROFILE:")
    for t in aset.annotations(d):
        print("    {} {}".format(t,hp.label(t)))
    print("  ICD ROLLUP PROFILE:")
    mterms = d2icd[d]
    for t in mterms:
        print("    {} {}".format(t,icd.label(t)))

show_phekb('OMIM:118450')
show_phekb('OMIM:610205')


TESTING: OMIM:118450 Alagille syndrome 1
  HPO PROFILE:
    HP:0001394 Cirrhosis
    HP:0001738 Exocrine pancreatic insufficiency
    HP:0001328 Specific learning disability
    HP:0000627 Posterior embryotoxon
    HP:0003189 Long nose
    HP:0004969 Peripheral pulmonary artery stenosis
    HP:0006571 Reduced number of intrahepatic bile ducts
    HP:0005280 Depressed nasal bridge
    HP:0001284 Areflexia
    HP:0002910 Elevated hepatic transaminases
    HP:0000482 Microcornea
    HP:0001680 Coarctation of aorta
    HP:0000400 Macrotia
    HP:0000585 Band keratopathy
    HP:0000076 Vesicoureteral reflux
    HP:0001256 Intellectual disability, mild
    HP:0000772 Abnormality of the ribs
    HP:0000533 Chorioretinal atrophy
    HP:0001492 Axenfeld anomaly
    HP:0000518 Cataract
    HP:0006579 Prolonged neonatal jaundice
    HP:0001297 Stroke
    HP:0000316 Hypertelorism
    HP:0000337 Broad forehead
    HP:0000545 Myopia
    HP:0001631 Atrial septal defect
    HP:0000089 Renal hypoplasi